[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# England vs Scotland: Covid-19 New Cases Dashboard

In [18]:
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [19]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [20]:
# Load JSON files and store the raw data in some variable. Edit as appropriate

with open("newCases.json", "rt") as INFILE:
    newCases=json.load(INFILE)


In [21]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=rawdata['data']
    
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()  
    def  parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    
    index=pd.date_range(startdate, enddate, freq='D')
    
    Englist = [England for England in datalist if England['areaName'] == 'England']
    Scotlist = [Scotland for Scotland in datalist if Scotland['areaName'] == 'Scotland']
    
    EngScotdf=pd.DataFrame(index=index, columns=['England', 'Scotland'])
    
    for entry in Englist: # each entry is a dictionary
        date=entry['date'] # our index position
        EngScotdf.loc[date, 'England']=entry['newCases']
    for entry in Scotlist: # each entry is a dictionary
        date=entry['date'] # our index position
        EngScotdf.loc[date, 'Scotland']=entry['newCases']
    
    

    EngScotdf.fillna(0.0, inplace=True)
    return EngScotdf


EngScotdf=wrangle_data(newCases)

In [22]:
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
        'areaType=nation'
    ]


    structure = {
        "areaName": "areaName",
        "date": "date",
        "newCases": "newCasesByPublishDate"
    }
    api = Cov19API(filters=filters, structure=structure)
    newCases=api.get_json()
    with open("newCases.json", "wt") as OUTF:
        json.dump(newCases, OUTF)
    with open("newCases.json", "rt") as INFILE:
        newCases=json.load(INFILE)
    return newCases


In [23]:
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    try:
        newCases=access_api()
        global EngScotdf
        EngScotdf=wrangle_data(newCases)
        refresh_graph()
        apibutton.button_style='success'
        apibutton.icon="check"
        print('Success!', end = '\r', flush = True)
    except:
        apibutton.button_style='danger'
        apibutton.icon="warning"
        print('Failed!', end = '\r', flush = True)
        
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='info',
    tooltip='Click to download current Public Health England data',
    icon='download'
)

apibutton.on_click(api_button_callback) 

display(apibutton)


Button(button_style='info', description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click t…

## Graphs and Analysis

In [24]:
nation=wdg.SelectMultiple(
    options=['England', 'Scotland'],
    value=('England', 'Scotland'),
    rows=2,
    description='Nation:',
    disabled=False
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

controls=wdg.HBox([nation, scale])

def newCases_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        EngScotdf[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

def refresh_graph():
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other 
    scale.value=current

graph=wdg.interactive_output(newCases_graph, {'gcols': nation, 'gscale': scale})

display(controls, graph)

Output()

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*